# Bonneville basin Landsat 8 timelapse 
## Adapted from Radwin, Bowen 2021
## Instructions:
## Run all cells and wait for annotated gif to appear. At first a link will appear, but ignore the link. Once the code is finished running two versions of the gif will be downloaded: one annotated and one plain. 

### Adjust the start/end dates as desired, but not earlier than 2013 as Landsat 8 was launched on 11 Feb, 2013
### Scenes are sorted by CLOUD_COVER but the CFMask algorithm has difficulties over the BB, so many scenes with low cloud scores are actually quite cloudy.

# Red = Halite, Green = Gypsum, and Blue = Carbonates (lacustrine)

In [222]:
##geemap.update_package()
import os
import ee
import geemap
import ipywidgets as widgets

In [223]:
import os
out_dir = os.path.join(os.path.expanduser("~"), 'Downloads')
if not os.path.exists(out_dir):
    os.makedirs(out_dir)

In [224]:
style = {'description_width': 'initial'}
start_year = widgets.IntSlider(description='Start Year:', value=2018, min=2013, max=2020, style=style)
end_year = widgets.IntSlider(description='End Year:', value=2020, min=2013, max=2021, style=style)
start_month = widgets.IntSlider(description='Start Month:', value=4, min=1, max=12, style=style)
end_month = widgets.IntSlider(description='End Month:', value=10, min=1, max=12, style=style)
cloud_thresh = widgets.IntSlider(description='Cloud % Threshold:', value=30, min=1, max=100, style=style)

In [225]:
years_box = widgets.HBox([start_year, end_year])
years_box

In [226]:
months_box = widgets.HBox([start_month, end_month])
months_box

In [227]:
cloud_thresh

IntSlider(value=30, description='Cloud % Threshold:', min=1, style=SliderStyle(description_width='initial'))

In [228]:
metadata_col = ee.ImageCollection("LANDSAT/LC08/C01/T1_SR").filter(ee.Filter.Or(
    ee.Filter.And(ee.Filter.eq('WRS_PATH', 39), ee.Filter.eq('WRS_ROW', 32)))) \
.filter(ee.Filter.calendarRange(start_year.value, end_year.value, 'year')) \
.filter(ee.Filter.calendarRange(start_month.value, end_month.value, 'month'))

In [229]:
output = widgets.Output()
Clip_Boundary = ee.Geometry.Polygon([[-114.183295, 39.631077], [-114.183295, 41.789335], [-112.699951, 41.789335],
                                     [-112.699951, 39.631077]])
LS_collection_initial = ee.ImageCollection("LANDSAT/LC08/C01/T1_SR").filter(ee.Filter.Or(
    ee.Filter.And(ee.Filter.eq('WRS_PATH', 39),
                  ee.Filter.eq('WRS_ROW', 32)),
    ee.Filter.And(ee.Filter.eq('WRS_PATH', 39),
                  ee.Filter.eq('WRS_ROW', 31)))) \
.filter(ee.Filter.calendarRange(start_year.value, end_year.value, 'year')) \
.filter(ee.Filter.calendarRange(start_month.value, end_month.value, 'month'))


In [230]:
low_cloud_col = ee.ImageCollection("LANDSAT/LC08/C01/T1_SR").filter(ee.Filter.Or(
    ee.Filter.And(ee.Filter.eq('WRS_PATH', 39),
                  ee.Filter.eq('WRS_ROW', 31)))).filter(ee.Filter.calendarRange(start_year.value, end_year.value, 'year')) \
.filter(ee.Filter.calendarRange(start_month.value, end_month.value, 'month')) \
.filter(ee.Filter.lt('CLOUD_COVER', cloud_thresh.value)).filter(ee.Filter.gt('CLOUD_COVER', 0))

low_cloud_col_list = low_cloud_col.toList(low_cloud_col.size())

def imagedate(image):
    datelist = ee.Image(image).date().format("YYYY-MM-dd")
    return datelist

low_cloud_dates = low_cloud_col_list.map(imagedate).distinct().sort()

In [231]:
##BEST METHOD FOR MOSAICING BY DATE, MAKES IMAGE ID THE DATE OF IMAGE ACQUISITION
def clp(image):
    clipped = image.clip(Clip_Boundary)
    return clipped ##Function for clipping the collection to the boundary

def mosaicByDate(imcol):
    imlist = imcol.toList(imcol.size())
    
    def imdate(im):
        return ee.Image(im).date().format("YYYY-MM-dd")
    
    unique_dates = imlist.map(imdate).distinct()
    
    def dater(d):
        d = ee.Date(d)
        im = imcol.filterDate(d, d.advance(1, "day")).mosaic()
        return im.set("system:time_start", d.millis(), 
        "system:id", d.format("YYYY-MM-dd"))
    
    mosaic_imlist = unique_dates.map(dater)
    
    return ee.ImageCollection(mosaic_imlist)

LS_collection = mosaicByDate(LS_collection_initial)

In [232]:
seq = ee.List.sequence(0, LS_collection.size().subtract(1))

def copyProps(index):
    source = ee.Image(metadata_col.toList(metadata_col.size()).get(index))
    dest = ee.Image(LS_collection.toList(LS_collection.size()).get(index))
    image = ee.Image(dest.copyProperties(source, properties=["CLOUD_COVER", "CLOUD_COVER_LAND", "SENSING_TIME",
                                                             "LANDSAT_ID", "system:time_start"]))
    return image

In [233]:
def simpledate(image):
    newtime = image.set('simple_date', ee.Date(image.date()).format('YYYY-MM-dd'))
    return newtime

In [234]:
LS_collection_metadata = ee.ImageCollection(seq.map(copyProps))
LS_collection_simple_dates = LS_collection_metadata.map(simpledate)
LS_preindices = LS_collection_simple_dates.filter(ee.Filter.inList("simple_date", low_cloud_dates))

In [235]:
def Hal_index(image):
    halite = image.expression(
    '(1.0*(RED - SWIR1))/(RED + SWIR1)', {
        'RED': image.select('B4'),
        'SWIR1': image.select('B6')
    })
    return halite

def h_band_rename(image):
    hband = image.select('constant').rename('h_index')
    return hband


halite_bands = LS_preindices.map(Hal_index).map(h_band_rename)

In [236]:
def Gyp_index(image):
    gypsum = image.expression(
    '1.0*(SWIR1-SWIR2)', {
        'SWIR1': image.select('B6'),
        'SWIR2': image.select('B7')
    })
    return gypsum

def g_band_rename(image):
    gband = image.select('constant').rename('g_index')
    return gband

gypsum_bands = LS_preindices.map(Gyp_index).map(g_band_rename)

In [237]:
def Carb_index(image):
    carbonates = image.expression(
    '(1.0*(1-(NIR-SWIR1))-(SWIR1-SWIR2)-RED-SWIR2)', {
        'RED': image.select('B4'),
        'NIR': image.select('B5'),
        'SWIR1': image.select('B6'),
        'SWIR2': image.select('B7')
    })
    return carbonates

def c_band_rename(image):
    cband = image.select('constant').rename('c_index')
    return cband

carbonate_bands = LS_preindices.map(Carb_index).map(c_band_rename)

In [238]:
##Combine indices with LS collection
com = LS_preindices.combine(halite_bands).combine(gypsum_bands).combine(carbonate_bands)
##print(com.aggregate_array('simple_date').sort().getInfo())

In [239]:
video_args = {
  'dimensions': 768,
  'region': Clip_Boundary,
  'framesPerSecond': 2,
    'bands': ['h_index', 'g_index', 'c_index'],
    'min': [0, 500, -12000],
    'max': [1.0, 2500, -3500],
    'crs': 'EPSG:3857'
}

In [240]:
work_dir = os.path.join(os.path.expanduser("~"), 'Downloads')
plain_gif_title = 'BB_Radwin_2021_indices_timelapse.gif'
out_gif = os.path.join(work_dir, plain_gif_title)

In [241]:
##Create gif
geemap.download_ee_video(com, video_args, out_gif)

Generating URL...
Please wait ...
The GIF image has been saved to: C:\Users\Mark Radwin\Downloads\BB_Radwin_2021_indices_timelapse.gif


In [242]:
##Make list of years from original LS collection for use in animations
def ymdList(col):
    def iter_func(image, newlist):
        date = image.date().format("YYYY-MM-dd");
        ##date = ee.Number.parse(image.date().format("YYYYMMdd"));
        newlist = ee.List(newlist);
        return ee.List(newlist.add(date).sort())
    ymd = col.iterate(iter_func, ee.List([]))
    return list(ee.List(ymd).reduce(ee.Reducer.frequencyHistogram()).getInfo().keys())
datess = ymdList(LS_preindices)
##print(datess)

In [243]:
##Add text to gif
annotated_gif_title = 'BB_Radwin_2021_indices_annotated_timelapse.gif'
texted_gif = os.path.join(work_dir, annotated_gif_title)

geemap.add_text_to_gif(out_gif, texted_gif, duration=500, 
                       xy=('2%', '2%'), text_sequence=datess, font_size=30, font_color='#ffffff')

label = 'Surface Changes in the \
Bonneville Basin'

geemap.add_text_to_gif(texted_gif, texted_gif, duration=500, 
                       xy=('2%', '94%'), text_sequence=label, font_size=20, font_color='#ffffff', progress_bar_color='white')


geemap.show_image(texted_gif)

Output()